###  Session Setup


In [ ]:
#@title 0.1: Install and Import Libraries
# Install the required libraries quietly.
!pip install transformers sentencepiece --quiet
print("✅ Libraries installed.")

# Import the necessary classes from the transformers library.
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
# 'textwrap' is a great tool for formatting our output nicely.
import textwrap
print("✅ Libraries imported successfully.")

✅ Libraries installed.
✅ Libraries imported successfully.


### Loading Our T5 Summarization Model


In [ ]:
#@title 1.2: Load the T5 Model and Tokenizer
# The model name we'll be using.
t5_model_name = 't5-base'

# The Tokenizer is responsible for converting text into a format the model understands.
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model_name)

# The Model is the pre-trained AI that performs the summarization.
t5_model = T5ForConditionalGeneration.from_pretrained(t5_model_name)

print(f"✅ T5 Model ('{t5_model_name}') is loaded and ready!")

✅ T5 Model ('t5-base') is loaded and ready!


###  The Summarizer Function


In [ ]:
#@title 1.3: Define the Core Summarizer Function
def generate_summary(text, min_len=40, max_len=120, beams=4):
    """
    Generates a high-quality abstractive summary for a given text using the T5 model.
    """
    # T5 models require a "summarize: " prefix to know which task to perform.
    input_text = "summarize: " + text.strip().replace("\n", " ")

    # Tokenize the text, ensuring it's not too long for the model.
    inputs = t5_tokenizer.encode(input_text, return_tensors='pt', max_length=1024, truncation=True)

    # Generate the summary using our specified parameters.
    summary_ids = t5_model.generate(
        inputs,
        max_length=max_len,
        min_length=min_len,
        num_beams=beams,
        no_repeat_ngram_size=3, # Prevents repeating phrases of 3 words.
        length_penalty=2.0,
        early_stopping=True
    )

    # Decode the result back into human-readable text.
    summary = t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

###  Code Examples with Diverse Texts


In [ ]:
#@title: Example 1 - Technical Text (Quantum Computing)
technical_text = """
Quantum computing is a revolutionary type of computation that harnesses the collective properties of quantum states,
such as superposition and entanglement, to perform calculations. While classical computers use bits that can be either
a 0 or a 1, a quantum computer uses qubits, which can be a 0, a 1, or both at the same time. This fundamental difference
allows quantum computers to solve complex problems that are intractable for even the most powerful classical supercomputers,
with potential applications in cryptography, materials science, and drug discovery.
"""

summary = generate_summary(technical_text, min_len=25, max_len=50)

print("----------- TECHNICAL TEXT -----------")
print(textwrap.fill(technical_text, width=100))
print("\n✨---------- T5 SUMMARY -----------✨")
print(textwrap.fill(summary, width=100))

In [ ]:
#@title: Example 2 - Business Text (Market Analysis)
business_text = """
The global market for renewable energy is projected to experience robust growth over the next decade. Key drivers include
increasing government incentives for clean energy, declining costs of solar and wind technologies, and growing consumer
awareness regarding climate change. However, challenges remain, such as the intermittency of renewable sources and the
need for significant grid infrastructure upgrades. Companies that can innovate in energy storage solutions and grid
management are best positioned to capitalize on this market trend.
"""

summary = generate_summary(business_text, min_len=30, max_len=90)

print("----------- BUSINESS TEXT -----------")
print(textwrap.fill(business_text, width=100))
print("\n✨---------- T5 SUMMARY -----------✨")
print(textwrap.fill(summary, width=100))

In [ ]:
#@title: Example 3 - Creative Text (Literary Description)
creative_text = """
The ancient library was a labyrinth of shadows and whispered knowledge. Sunlight struggled through a high,
arched window, illuminating motes of dust that danced like tiny sprites in the golden shafts of light.
The air smelled of aging paper, leather, and a faint, sweet hint of vanilla. Every towering bookshelf
was a gateway to another world, each leather-bound volume a silent promise of adventure, history,
or forgotten magic. It was a place where time itself seemed to slow down, holding its breath in reverence
for the stories it contained.
"""

summary = generate_summary(creative_text, min_len=20, max_len=70)

print("----------- CREATIVE TEXT -----------")
print(textwrap.fill(creative_text, width=100))
print("\n✨---------- T5 SUMMARY -----------✨")
print(textwrap.fill(summary, width=100))

### The Interactive Summarizer Studio


In [ ]:
#@title: Your Interactive Summarizer Studio! ⚡️
#@markdown ### 👈 Paste your text below and tune the parameters!
input_text = 'The James Webb Space Telescope (JWST) is a space telescope designed primarily to conduct infrared astronomy. As the largest optical telescope in space, its high resolution and sensitivity allow it to view objects too old, distant, or faint for the Hubble Space Telescope. This has enabled investigations in many fields of astronomy and cosmology, such as observation of the first stars, the formation of the first galaxies, and detailed atmospheric characterization of potentially habitable exoplanets. The U.S. National Aeronautics and Space Administration (NASA) led JWST\'s development in collaboration with the European Space Agency (ESA) and the Canadian Space Agency (CSA).' #@param {type:"string"}
min_length = 45 #@param {type:"slider", min:10, max:100, step:5}
max_length = 140 #@param {type:"slider", min:50, max:200, step:10}
num_beams = 5 #@param {type:"slider", min:2, max:8, step:1}

# --- Run the summarizer with your settings ---
generated_summary = generate_summary(input_text, min_len=min_length, max_len=max_length, beams=num_beams)

# --- Display the results and analysis ---
original_word_count = len(input_text.split())
summary_word_count = len(generated_summary.split())
reduction = 100 - (summary_word_count / original_word_count * 100)

print("----------- YOUR INPUT TEXT -----------")
print(textwrap.fill(input_text, width=100))
print("\n✨---------- GENERATED SUMMARY -----------✨")
print(textwrap.fill(generated_summary, width=100))
print("\n📊---------- ANALYSIS -----------📊")
print(f"Original Word Count: {original_word_count}")
print(f"Summary Word Count: {summary_word_count}")
print(f"Text Reduction: {reduction:.1f}%")

###  The Paraphraser Function




In [ ]:
#@title : Paraphraser Function
def generate_paraphrases(text, num_return=5, beams=10):
    """
    Generates multiple high-quality paraphrases for a given text using the PEGASUS model.
    """
    # Tokenize the input text.
    inputs = pegasus_tokenizer.encode(text, return_tensors='pt', truncation=True)

    # Generate the paraphrases using beam search.
    paraphrase_ids = pegasus_model.generate(
        inputs,
        max_length=60,
        num_beams=beams,
        num_return_sequences=num_return,
        early_stopping=True
    )

    # Decode the results back into text.
    paraphrases = pegasus_tokenizer.batch_decode(paraphrase_ids, skip_special_tokens=True)
    return paraphrases

###  Code Examples with Diverse Sentences


In [ ]:
#@title: Example 1 - Formal / Academic Sentence
formal_sentence = "The empirical data indicates a statistically significant correlation between the two variables."
paraphrases = generate_paraphrases(formal_sentence, num_return=4)

print(f"----------- ORIGINAL FORMAL SENTENCE -----------\n'{formal_sentence}'\n")
print("🤖---------- PEGASUS PARAPHRASES ----------🤖")
for i, p in enumerate(paraphrases):
    print(f"  {i+1}. {p}")

In [ ]:
#@title: Example 2 - Casual / Idiomatic Sentence
casual_sentence = "To be honest, that new project is a real pain in the neck."
paraphrases = generate_paraphrases(casual_sentence, num_return=4)

print(f"----------- ORIGINAL CASUAL SENTENCE -----------\n'{casual_sentence}'\n")
print("🤖---------- PEGASUS PARAPHRASES ----------🤖")
for i, p in enumerate(paraphrases):
    print(f"  {i+1}. {p}")

In [ ]:
#@title: Example 3 - Marketing Call-to-Action
marketing_sentence = "Don't miss out on our exclusive offer – shop now to save 50%!"
paraphrases = generate_paraphrases(marketing_sentence, num_return=4)

print(f"----------- ORIGINAL MARKETING SENTENCE -----------\n'{marketing_sentence}'\n")
print("🤖---------- PEGASUS PARAPHRASES ----------🤖")
for i, p in enumerate(paraphrases):
    print(f"  {i+1}. {p}")

### Section 2.5: The Interactive Paraphraser Playground


In [ ]:
#@title: Your Interactive Paraphraser Playground! ⚡️
#@markdown ### 👈 Type your sentence and choose your settings!
input_sentence = "Learning new skills is essential for career growth." #@param {type:"string"}
num_paraphrases = 5 #@param {type:"slider", min:1, max:10, step:1}
quality_vs_speed_beams = 9 #@param {type:"slider", min:2, max:15, step:1}

# --- Run the paraphraser with your settings ---
generated_paraphrases = generate_paraphrases(input_sentence, num_return=num_paraphrases, beams=quality_vs_speed_beams)

# --- Display the results ---
print(f"----------- ORIGINAL SENTENCE -----------\n'{input_sentence}'\n")
print(f"🤖---------- {len(generated_paraphrases)} GENERATED PARAPHRASES (Quality: {quality_vs_speed_beams}) ----------🤖")
for i, p in enumerate(generated_paraphrases):
    print(f"  {i+1}. {p}")